In [1]:
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import random

# Définir un environnement simple avec des agents
class SimpleEnv:
    def __init__(self, num_agents):
        self.num_agents = num_agents
        self.positions = torch.rand((num_agents, 2))  # Positions initiales aléatoires des agents
        self.velocities = torch.rand((num_agents, 2))  # Vitesse initiale aléatoire des agents
        self.time_step = 0.1  # Pas de temps pour la simulation

    def reset(self):
        self.positions = torch.rand((self.num_agents, 2))  # Réinitialiser les positions
        self.velocities = torch.rand((self.num_agents, 2))  # Réinitialiser les vitesses
        return self.positions, self.velocities

    def step(self, actions):
        # Mettre à jour les positions et vitesses des agents selon leurs actions
        self.velocities += actions * self.time_step  # actions: déplacement des agents
        self.positions += self.velocities * self.time_step  # Mettre à jour les positions

        # Calculer la récompense en fonction de la proximité des agents
        reward = self.compute_reward()
        return self.positions, self.velocities, reward

    def compute_reward(self):
        reward = 0
        for i in range(self.num_agents):
            for j in range(i+1, self.num_agents):
                distance = torch.norm(self.positions[i] - self.positions[j])
                reward -= distance  # Plus les agents sont proches, plus la récompense est élevée
        return reward

# Définir le modèle GNN
class SimpleGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = torch.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Initialisation de l'environnement
num_agents = 3
env = SimpleEnv(num_agents)

# Initialisation du modèle GNN
input_dim = 2  # Nombre de caractéristiques (position, vitesse)
hidden_dim = 4
output_dim = 2  # Actions (déplacements des agents)
model = SimpleGNN(input_dim, hidden_dim, output_dim)

# Fonction d'entraînement du modèle
def train(env, model, num_episodes=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for episode in range(num_episodes):
        positions, velocities = env.reset()

        # Préparer les caractéristiques des nœuds (agents)
        x = torch.cat((positions, velocities), dim=1)  # Concatenate positions et vitesses

        # Créer un graphe (arêtes entre agents proches)
        edge_index = create_edge_index(num_agents)

        # Calculer les actions (déplacements) avec le modèle GNN
        actions = model(x, edge_index)

        # Faire un pas dans l'environnement
        new_positions, new_velocities, reward = env.step(actions)

        # Calculer la perte (nous voulons maximiser la récompense)
        loss = -reward  # Le but est de maximiser la récompense, donc perte négative

        # Rétropropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if episode % 10 == 0:
            print(f"Episode {episode}, Reward: {reward.item()}")

# Créer un graphe basé sur la proximité des agents
def create_edge_index(num_agents):
    edge_index = []
    for i in range(num_agents):
        for j in range(i+1, num_agents):
            edge_index.append([i, j])
            edge_index.append([j, i])  # Ajouter les deux directions (i -> j et j -> i)
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Entraîner le modèle
train(env, model)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x4 and 2x4)

In [4]:
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import random

# Définir un environnement simple avec des agents
class SimpleEnv:
    def __init__(self, num_agents):
        self.num_agents = num_agents
        self.positions = torch.rand((num_agents, 2))  # Positions initiales aléatoires des agents
        self.velocities = torch.rand((num_agents, 2))  # Vitesse initiale aléatoire des agents
        self.time_step = 0.1  # Pas de temps pour la simulation

    def reset(self):
        self.positions = torch.rand((self.num_agents, 2))  # Réinitialiser les positions
        self.velocities = torch.rand((self.num_agents, 2))  # Réinitialiser les vitesses
        return self.positions, self.velocities

    def step(self, actions):
        # Mettre à jour les positions et vitesses des agents selon leurs actions
        self.velocities += actions * self.time_step  # actions: déplacement des agents
        self.positions += self.velocities * self.time_step  # Mettre à jour les positions

        # Calculer la récompense en fonction de la proximité des agents
        reward = self.compute_reward()
        return self.positions, self.velocities, reward

    def compute_reward(self):
        reward = 0
        for i in range(self.num_agents):
            for j in range(i+1, self.num_agents):
                distance = torch.norm(self.positions[i] - self.positions[j])
                reward -= distance  # Plus les agents sont proches, plus la récompense est élevée
        return reward

# Définir le modèle GNN
class SimpleGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = torch.relu(self.conv1(x, edge_index))  # Appliquer la première convolution
        x = self.conv2(x, edge_index)  # Appliquer la deuxième convolution
        return x

# Initialisation de l'environnement
num_agents = 3
env = SimpleEnv(num_agents)

# Initialisation du modèle GNN
input_dim = 4  # Nombre de caractéristiques (position, vitesse) concaténées
hidden_dim = 4
output_dim = 2  # Actions (déplacements des agents)
model = SimpleGNN(input_dim, hidden_dim, output_dim)

# Fonction d'entraînement du modèle
def train(env, model, num_episodes=1000):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for episode in range(num_episodes):
        positions, velocities = env.reset()

        # Préparer les caractéristiques des nœuds (agents)
        x = torch.cat((positions, velocities), dim=1)  # Concatenate positions et vitesses (4 caractéristiques)

        # Créer un graphe (arêtes entre agents proches)
        edge_index = create_edge_index(num_agents)

        # Calculer les actions (déplacements) avec le modèle GNN
        actions = model(x, edge_index)

        # Faire un pas dans l'environnement
        new_positions, new_velocities, reward = env.step(actions)

        # Calculer la perte (nous voulons maximiser la récompense)
        loss = -reward  # Le but est de maximiser la récompense, donc perte négative

        # Rétropropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if episode % 10 == 0:
            print(f"Episode {episode}, Reward: {reward.item()}")

# Créer un graphe basé sur la proximité des agents
def create_edge_index(num_agents):
    edge_index = []
    for i in range(num_agents):
        for j in range(i+1, num_agents):
            edge_index.append([i, j])
            edge_index.append([j, i])  # Ajouter les deux directions (i -> j et j -> i)
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Entraîner le modèle
train(env, model)


Episode 0, Reward: -1.6426904201507568
Episode 10, Reward: -1.1354026794433594
Episode 20, Reward: -0.9518758058547974
Episode 30, Reward: -1.6432170867919922
Episode 40, Reward: -1.60065758228302
Episode 50, Reward: -2.17828631401062
Episode 60, Reward: -1.8209532499313354
Episode 70, Reward: -1.0061274766921997
Episode 80, Reward: -1.5181645154953003
Episode 90, Reward: -1.7908287048339844
Episode 100, Reward: -1.4913365840911865
Episode 110, Reward: -1.433598279953003
Episode 120, Reward: -1.876486897468567
Episode 130, Reward: -1.562821626663208
Episode 140, Reward: -1.9320898056030273
Episode 150, Reward: -1.3887519836425781
Episode 160, Reward: -0.6437910795211792
Episode 170, Reward: -1.7163625955581665
Episode 180, Reward: -1.241245985031128
Episode 190, Reward: -1.543684959411621
Episode 200, Reward: -1.1578516960144043
Episode 210, Reward: -0.32796913385391235
Episode 220, Reward: -1.3898110389709473
Episode 230, Reward: -1.0210245847702026
Episode 240, Reward: -1.29523205757